In [1]:
#Import the necessary libraries for navigating the US Census API
import requests
import pandas as pd
import ast
import re
import numpy as np

In [2]:
#Creates a dictionary of codes and corresponding titles for the 2011 American Community Survey
census_codes_2011 = {
    'Total_Pop': 'B01003_001E',
    'Total_Males': 'B01001_002E',
    'Total_Females': 'B01001_026E',
    'Median_Age': 'B01002_001E',
    'Total_Veterans': 'B21001_002E',
    'Total_White_People': 'B02001_002E',
    'Total_Afr_Am_People': 'B02001_003E',
    'Total_Asian': 'B02001_005E',
    'Total_American_Ind_Alk_Ntv': 'B02001_004E',
    'Total_Cuban_Origin': 'C03001_006E',
    'Total_Puerto_Rican_Origin':'C03001_005E',
    'Total_Dominican_Origin':'C03001_007E',
    'Total_Mexican_Origin': 'C03001_004E',
    'Total_Native_US':'B05001_002E',
    'Total_Non_Citizens': 'B05001_006E',
    'Total_Foreign_Born': 'B05012_003E',
    'Total_Children': 'B09001_001E',
    'Total_Married_Households': 'B11001_003E',
    'Total_Less_Highschool': 'B06009_002E',
    'Total_Bachelor_Holders': 'B06009_005E',
    'Total_STEM': 'B15012_009E',
    'Total_Business_Degree': 'B15012_010E',
    'Total_Labor_Force_Eligible': 'B23025_003E', #code changed in 2015
    'Total_Employed': 'B23025_004E',
    'Total_Unemployed': 'B23025_005E',
    'Median_Household_Income': 'B19013_001E',
    'Median_Income': 'B06011_001E',
    'Below_Poverty_Level_LTM': 'B17001_002E',
    'Total_Wealthy_Households': 'B19001_017E', #Earns $200k or more
    'Gini_Index': 'B19083_001E',
    'Median_Age_Of_Worker': 'B23013_001E',
    'Total_Workers': 'B24050_001E',
    'Total_Working_Agricult': 'B24050_002E',
    'Total_Working_Construction': 'B24050_005E',
    'Total_Working_Manufacturing': 'B24050_006E',
    'Total_Working_Retail': 'B24050_008E',
    'Total_Working_Transportation': 'B24050_009E',
    'Total_Working_Finance': 'B24050_013E',
    'Total_Working_Education': 'B24050_021E',
    'Total_Working_Health': 'B24050_022E',
    'Total_Working_Food': 'B24050_025E',
    'Total_Working_Public_Admin': 'B24050_027E',
    'Total_Working_Information': 'B24050_012E',
    'Median_Gross_Rent': 'B25064_001E'
   
}

#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes_2011 = {v: k for k, v in census_codes_2011.items()}
inv_census_codes_2011.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})

#Creates a string of codes to be used in the API call
columns_url_2011 = ''

for key in census_codes_2011:
    columns_url_2011 += census_codes_2011[key] + ','
    
columns_url_2011

'B01003_001E,B01001_002E,B01001_026E,B01002_001E,B21001_002E,B02001_002E,B02001_003E,B02001_005E,B02001_004E,C03001_006E,C03001_005E,C03001_007E,C03001_004E,B05001_002E,B05001_006E,B05012_003E,B09001_001E,B11001_003E,B06009_002E,B06009_005E,B15012_009E,B15012_010E,B23025_003E,B23025_004E,B23025_005E,B19013_001E,B06011_001E,B17001_002E,B19001_017E,B19083_001E,B23013_001E,B24050_001E,B24050_002E,B24050_005E,B24050_006E,B24050_008E,B24050_009E,B24050_013E,B24050_021E,B24050_022E,B24050_025E,B24050_027E,B24050_012E,B25064_001E,'

In [3]:
#Creates a dictionary of codes and corresponding titles for the 2015 American Community Survey
census_codes_2015 = {
    'Total_Pop': 'B01003_001E',
    'Total_Males': 'B01001_002E',
    'Total_Females': 'B01001_026E',
    'Median_Age': 'B01002_001E',
    'Total_Veterans': 'B21001_002E',
    'Total_White_People': 'B02001_002E',
    'Total_Afr_Am_People': 'B02001_003E',
    'Total_Asian': 'B02001_005E',
    'Total_American_Ind_Alk_Ntv': 'B02001_004E',
    'Total_Cuban_Origin': 'C03001_006E',
    'Total_Puerto_Rican_Origin':'C03001_005E',
    'Total_Dominican_Origin':'C03001_007E',
    'Total_Mexican_Origin': 'C03001_004E',
    'Total_Native_US':'B05001_002E',
    'Total_Non_Citizens': 'B05001_006E',
    'Total_Foreign_Born': 'B05012_003E',
    'Total_Children': 'B09001_001E',
    'Total_Married_Households': 'B11001_003E',
    'Total_Less_Highschool': 'B06009_002E',
    'Total_Bachelor_Holders': 'B06009_005E',
    'Total_STEM': 'B15012_009E',
    'Total_Business_Degree': 'B15012_010E',
    'Total_Labor_Force_Eligible': 'B28007_002E', #code changed in 2015
    'Total_Employed': 'B28007_003E',
    'Total_Unemployed': 'B28007_009E',
    'Median_Household_Income': 'B19013_001E',
    'Median_Income': 'B06011_001E',
    'Below_Poverty_Level_LTM': 'B17001_002E',
    'Total_Wealthy_Households': 'B19001_017E', #Earns $200k or more
    'Gini_Index': 'B19083_001E',
    'Median_Age_Of_Worker': 'B23013_001E',
    'Total_Workers': 'B24050_001E',
    'Total_Working_Agricult': 'B24050_002E',
    'Total_Working_Construction': 'B24050_005E',
    'Total_Working_Manufacturing': 'B24050_006E',
    'Total_Working_Retail': 'B24050_008E',
    'Total_Working_Transportation': 'B24050_009E',
    'Total_Working_Finance': 'B24050_013E',
    'Total_Working_Education': 'B24050_021E',
    'Total_Working_Health': 'B24050_022E',
    'Total_Working_Food': 'B24050_025E',
    'Total_Working_Public_Admin': 'B24050_027E',
    'Total_Working_Information': 'B24050_012E',
    'Median_Gross_Rent': 'B25031_001E'
   
}

#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes_2015 = {v: k for k, v in census_codes_2015.items()}
inv_census_codes_2015.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})


#Creates a string of codes to be used in the API call
columns_url_2015 = ''

for key in census_codes_2015:
    columns_url_2015 += census_codes_2015[key] + ','
    
columns_url_2015

'B01003_001E,B01001_002E,B01001_026E,B01002_001E,B21001_002E,B02001_002E,B02001_003E,B02001_005E,B02001_004E,C03001_006E,C03001_005E,C03001_007E,C03001_004E,B05001_002E,B05001_006E,B05012_003E,B09001_001E,B11001_003E,B06009_002E,B06009_005E,B15012_009E,B15012_010E,B28007_002E,B28007_003E,B28007_009E,B19013_001E,B06011_001E,B17001_002E,B19001_017E,B19083_001E,B23013_001E,B24050_001E,B24050_002E,B24050_005E,B24050_006E,B24050_008E,B24050_009E,B24050_013E,B24050_021E,B24050_022E,B24050_025E,B24050_027E,B24050_012E,B25031_001E,'

In [4]:
#A function for the API call
def obtain_census_data(year, codes):
    congressional_district_url = 'https://api.census.gov/data/{}/acs/acs1?get={}NAME&for=congressional%20district:*&in=state:*'.format(year, codes)
    congressional_district_content = requests.get(congressional_district_url)
    congressional_district_contenttext = congressional_district_content.text.replace('\n', '').replace('null', '"Replace"')
    congressional_district_information_list = ast.literal_eval(congressional_district_contenttext)
    print(len(congressional_district_information_list))
    return congressional_district_information_list
    

In [5]:
#A function to turn the inner list from the API call into a dictionary
#The dictionary will represent a district and its census info
def turn_info_list_to_dict(keys_list, list_of_cd_info):
    return dict(zip(keys_list, list_of_cd_info))

In [6]:
#A function that takes in the list of lists and returns a list of dictionaries
def congressional_district_dictionary_list(list_of_cds_info):
    keys_list = list_of_cds_info[0]
    cd_dictionary_list = []
    
    for district in list_of_cds_info[1:]:
        cd_dictionary_list.append(turn_info_list_to_dict(keys_list, district))
    
    return cd_dictionary_list

In [7]:
#The code to obtain the 2011 data that will be used to predict the 2012 results 
congressional_data_2012 = congressional_district_dictionary_list(obtain_census_data('2011', columns_url_2011))
congressional_data_2012

438


[{'B01003_001E': '693871',
  'B01001_002E': '334016',
  'B01001_026E': '359855',
  'B01002_001E': '38.6',
  'B21001_002E': '60234',
  'B02001_002E': '469202',
  'B02001_003E': '195366',
  'B02001_005E': '9227',
  'B02001_004E': '7115',
  'C03001_006E': '376',
  'C03001_005E': '3238',
  'C03001_007E': '0',
  'C03001_004E': '9580',
  'B05001_002E': '668746',
  'B05001_006E': '11078',
  'B05012_003E': '21492',
  'B09001_001E': '167610',
  'B11001_003E': '126773',
  'B06009_002E': '69476',
  'B06009_005E': '64308',
  'B15012_009E': '12373',
  'B15012_010E': '21819',
  'B23025_003E': '321306',
  'B23025_004E': '284910',
  'B23025_005E': '36396',
  'B19013_001E': '43144',
  'B06011_001E': '23254',
  'B17001_002E': '126778',
  'B19001_017E': '5588',
  'B19083_001E': '0.4623',
  'B23013_001E': '40.9',
  'B24050_001E': '284910',
  'B24050_002E': '4287',
  'B24050_005E': '21591',
  'B24050_006E': '31112',
  'B24050_008E': '40290',
  'B24050_009E': '15519',
  'B24050_013E': '15615',
  'B24050_021

In [8]:
#The code to obtain the 2015 data that will be used to predict the 2016 results 
congressional_data_2016 = congressional_district_dictionary_list(obtain_census_data('2015', columns_url_2015))
congressional_data_2016

438


[{'B01003_001E': '711436',
  'B01001_002E': '341326',
  'B01001_026E': '370110',
  'B01002_001E': '41.0',
  'B21001_002E': '31058',
  'B02001_002E': '497769',
  'B02001_003E': '110721',
  'B02001_005E': '34271',
  'B02001_004E': '2917',
  'C03001_006E': '3134',
  'C03001_005E': '76479',
  'C03001_007E': '9107',
  'C03001_004E': '4916',
  'B05001_002E': '564110',
  'B05001_006E': '51998',
  'B05012_003E': '114085',
  'B09001_001E': '147905',
  'B11001_003E': '127785',
  'B06009_002E': '54796',
  'B06009_005E': '106398',
  'B15012_009E': '17383',
  'B15012_010E': '34714',
  'B28007_002E': '387730',
  'B28007_003E': '360565',
  'B28007_009E': '27165',
  'B19013_001E': '69959',
  'B06011_001E': '35421',
  'B17001_002E': '74486',
  'B19001_017E': '22506',
  'B19083_001E': '0.4569',
  'B23013_001E': '41.8',
  'B24050_001E': '361863',
  'B24050_002E': '961',
  'B24050_005E': '18557',
  'B24050_006E': '38501',
  'B24050_008E': '36234',
  'B24050_009E': '16206',
  'B24050_013E': '40602',
  'B24

In [9]:
#A dictionary of state codes and abbreviations to relate the tables that use either
#the abbreviation or full state name
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

#A dictionary to invert the above dictionary
inv_state = {v: k for k, v in states.items()}

In [10]:
#The following code creates a pandas dataframe from the 2011 API call then renames the
#columns to include underscores. A unique identifier for each district and election
#year is created
congressional_df_2012 = pd.DataFrame(congressional_data_2012)
congressional_df_2012.rename(columns = {'state': 'State_Id', 'NAME': 'District_Name', 'congressional district': 'CD'}, inplace = True)
congressional_df_2012['State'] = congressional_df_2012.District_Name.apply(lambda x: x.split(', ')[1])
congressional_df_2012['State'] = congressional_df_2012['State'].map(inv_state)
congressional_df_2012.loc[congressional_df_2012['CD'] == '00', 'CD'] = 'AL'
congressional_df_2012["CD_Id_Year"] = congressional_df_2012["State"] + '-' + congressional_df_2012["CD"] + '-2012'
congressional_df_2012.columns = congressional_df_2012.columns.to_series().map(inv_census_codes_2011)
congressional_df_2012.head()

,Total_Males,Total_Females,Median_Age,Total_Pop,Total_White_People,Total_Afr_Am_People,Total_American_Ind_Alk_Ntv,Total_Asian,Total_Native_US,Total_Non_Citizens,...,Median_Gross_Rent,Total_Mexican_Origin,Total_Puerto_Rican_Origin,Total_Cuban_Origin,Total_Dominican_Origin,District_Name,CD,State_Id,State,CD_Id_Year
0,334016,359855,38.6,693871,469202,195366,7115,9227,668746,11078,...,755,9580,3238,376,0,"Congressional District 1 (112th Congress), Ala...",01,01,AL,AL-01-2012
1,327723,347811,37.7,675534,433488,215736,2849,5064,652245,10800,...,662,13471,2046,262,217,"Congressional District 2 (112th Congress), Ala...",02,01,AL,AL-02-2012
2,332280,351660,36.9,683940,440065,220342,1270,9253,656872,12616,...,663,11750,2137,666,0,"Congressional District 3 (112th Congress), Ala...",03,01,AL,AL-03-2012
3,323747,339525,40.9,663272,598410,36302,2731,2552,633455,20127,...,558,31511,776,627,311,"Congressional District 4 (112th Congress), Ala...",04,01,AL,AL-04-2012
4,355468,369354,39.4,724822,548779,132512,7204,9512,688743,15015,...,647,23212,2820,395,434,"Congressional District 5 (112th Congress), Ala...",05,01,AL,AL-05-2012


In [11]:
congressional_df_2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 49 columns):
Total_Males                     437 non-null object
Total_Females                   437 non-null object
Median_Age                      437 non-null object
Total_Pop                       437 non-null object
Total_White_People              437 non-null object
Total_Afr_Am_People             437 non-null object
Total_American_Ind_Alk_Ntv      437 non-null object
Total_Asian                     437 non-null object
Total_Native_US                 437 non-null object
Total_Non_Citizens              437 non-null object
Total_Foreign_Born              437 non-null object
Total_Less_Highschool           437 non-null object
Total_Bachelor_Holders          437 non-null object
Median_Income                   437 non-null object
Total_Children                  437 non-null object
Total_Married_Households        437 non-null object
Total_STEM                      437 non-null object
Total_Busin

In [12]:
#The following code creates a pandas dataframe from the 2015 API call then renames the
#columns to include underscores. A unique identifier for each district and election
#year is created
congressional_df_2016 = pd.DataFrame(congressional_data_2016)
congressional_df_2016.rename(columns = {'state': 'State_Id', 'NAME': 'District_Name', 'congressional district': 'CD'}, inplace = True)
congressional_df_2016['State'] = congressional_df_2016.District_Name.apply(lambda x: x.split(', ')[1])
congressional_df_2016['State'] = congressional_df_2016['State'].map(inv_state)
congressional_df_2016.loc[congressional_df_2016['CD'] == '00', 'CD'] = 'AL'
congressional_df_2016["CD_Id_Year"] = congressional_df_2016["State"] + '-' + congressional_df_2016["CD"] + '-2016'
congressional_df_2016.columns = congressional_df_2016.columns.to_series().map(inv_census_codes_2015)
congressional_df_2016.head()

,Total_Males,Total_Females,Median_Age,Total_Pop,Total_White_People,Total_Afr_Am_People,Total_American_Ind_Alk_Ntv,Total_Asian,Total_Native_US,Total_Non_Citizens,...,Total_Unemployed,Total_Mexican_Origin,Total_Puerto_Rican_Origin,Total_Cuban_Origin,Total_Dominican_Origin,District_Name,CD,State_Id,State,CD_Id_Year
0,341326,370110,41.0,711436,497769,110721,2917,34271,564110,51998,...,27165,4916,76479,3134,9107,"Congressional District 1 (114th Congress), Con...",01,09,CT,CT-01-2016
1,356636,353011,41.4,709647,605885,29651,2989,22990,643287,22962,...,23360,9001,35905,1147,3467,"Congressional District 2 (114th Congress), Con...",02,09,CT,CT-02-2016
2,347519,372974,39.0,720493,526161,95429,1472,33050,601405,51568,...,25377,18343,54876,2714,4373,"Congressional District 3 (114th Congress), Con...",03,09,CT,CT-03-2016
3,356360,380382,39.7,736742,537381,95726,404,39399,551846,84805,...,29228,17883,47053,3115,7721,"Congressional District 4 (114th Congress), Con...",04,09,CT,CT-04-2016
4,350222,362346,41.8,712568,580747,49044,1111,26539,581569,54810,...,26105,4875,65757,2649,13379,"Congressional District 5 (114th Congress), Con...",05,09,CT,CT-05-2016


In [13]:
#This cell concats the 2012 election data and 2016 election data
congressional_df = pd.concat([congressional_df_2016,congressional_df_2012], sort=False)
congressional_df

,Total_Males,Total_Females,Median_Age,Total_Pop,Total_White_People,Total_Afr_Am_People,Total_American_Ind_Alk_Ntv,Total_Asian,Total_Native_US,Total_Non_Citizens,...,Total_Unemployed,Total_Mexican_Origin,Total_Puerto_Rican_Origin,Total_Cuban_Origin,Total_Dominican_Origin,District_Name,CD,State_Id,State,CD_Id_Year
0,341326,370110,41.0,711436,497769,110721,2917,34271,564110,51998,...,27165,4916,76479,3134,9107,"Congressional District 1 (114th Congress), Con...",01,09,CT,CT-01-2016
1,356636,353011,41.4,709647,605885,29651,2989,22990,643287,22962,...,23360,9001,35905,1147,3467,"Congressional District 2 (114th Congress), Con...",02,09,CT,CT-02-2016
2,347519,372974,39.0,720493,526161,95429,1472,33050,601405,51568,...,25377,18343,54876,2714,4373,"Congressional District 3 (114th Congress), Con...",03,09,CT,CT-03-2016
3,356360,380382,39.7,736742,537381,95726,404,39399,551846,84805,...,29228,17883,47053,3115,7721,"Congressional District 4 (114th Congress), Con...",04,09,CT,CT-04-2016
4,350222,362346,41.8,712568,580747,49044,1111,26539,581569,54810,...,26105,4875,65757,2649,13379,"Congressional District 5 (114th Congress), Con...",05,09,CT,CT-05-2016
5,458208,487726,39.7,945934,654904,203863,3121,36114,845668,47986,...,27307,34785,21476,1638,4443,Congressional District (at Large) (114th Congr...,AL,10,DE,DE-AL-2016
6,319705,352523,33.8,672228,269143,318831,1661,25944,564275,53472,...,26171,12653,6989,4021,2354,"Delegate District (at Large) (114th Congress),...",98,11,DC,DC-98-2016
7,379933,370995,38.5,750928,578965,98282,2120,17017,690667,17691,...,25319,18729,12726,3442,447,"Congressional District 1 (114th Congress), Flo...",01,12,FL,FL-01-2016
8,361387,356786,36.5,718173,497970,179656,2603,14334,664299,21475,...,27159,15057,9967,5849,645,"Congressional District 2 (114th Congress), Flo...",02,12,FL,FL-02-2016
9,358340,362765,37.9,721105,569233,98797,2150,23880,653621,25425,...,25147,17273,16856,8247,1889,"Congressional District 3 (114th Congress), Flo...",03,12,FL,FL-03-2016


In [14]:
#This code imports the data for the previous presidential elections and renames the columns
#A unique identifier for each district and election year is created
election_results = pd.read_csv('election_results.csv')
election_results['State_AB'] = election_results.CD.apply(lambda x: x.split('-')[0])
election_results.rename(columns = {'CD': 'CD_Id'}, inplace = True)
election_results["CD_Id_Year"] = election_results['CD_Id'] + '-' + election_results['Year'].astype(str)
election_results.drop(['CD_Id', 'Year', 'State_AB'], axis = 1, inplace = True)
election_results = election_results[['CD_Id_Year', 'Previous_Presidential_D', 'Previous_Presidential_R', 'Target_Presidential_D', 'Target_Presidential_R']]
election_results

,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
0,DC-98-2016,90.91,7.28,90.86,4.09
1,AK-AL-2016,41.20,55.30,37.60,52.80
2,AL-01-2016,37.40,61.80,34.10,63.50
3,AL-02-2016,36.40,62.90,33.00,64.90
4,AL-03-2016,36.80,62.30,32.30,65.30
5,AL-04-2016,24.00,74.80,17.40,80.40
6,AL-05-2016,34.90,63.90,31.30,64.70
7,AL-06-2016,24.70,74.30,26.10,70.80
8,AL-07-2016,72.40,27.10,69.80,28.60
9,AR-01-2016,36.30,61.00,30.20,65.00


In [15]:
#This code combines the census data and election results
election_census = pd.merge(congressional_df, election_results, on='CD_Id_Year', how = 'left')
election_census

,Total_Males,Total_Females,Median_Age,Total_Pop,Total_White_People,Total_Afr_Am_People,Total_American_Ind_Alk_Ntv,Total_Asian,Total_Native_US,Total_Non_Citizens,...,Total_Dominican_Origin,District_Name,CD,State_Id,State,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
0,341326,370110,41.0,711436,497769,110721,2917,34271,564110,51998,...,9107,"Congressional District 1 (114th Congress), Con...",01,09,CT,CT-01-2016,63.30,35.60,59.40,36.30
1,356636,353011,41.4,709647,605885,29651,2989,22990,643287,22962,...,3467,"Congressional District 2 (114th Congress), Con...",02,09,CT,CT-02-2016,55.90,42.60,48.70,45.80
2,347519,372974,39.0,720493,526161,95429,1472,33050,601405,51568,...,4373,"Congressional District 3 (114th Congress), Con...",03,09,CT,CT-03-2016,62.60,36.30,55.90,40.40
3,356360,380382,39.7,736742,537381,95726,404,39399,551846,84805,...,7721,"Congressional District 4 (114th Congress), Con...",04,09,CT,CT-04-2016,55.10,44.00,59.60,36.60
4,350222,362346,41.8,712568,580747,49044,1111,26539,581569,54810,...,13379,"Congressional District 5 (114th Congress), Con...",05,09,CT,CT-05-2016,53.50,45.30,49.90,45.80
5,458208,487726,39.7,945934,654904,203863,3121,36114,845668,47986,...,4443,Congressional District (at Large) (114th Congr...,AL,10,DE,DE-AL-2016,58.60,40.00,53.40,41.90
6,319705,352523,33.8,672228,269143,318831,1661,25944,564275,53472,...,2354,"Delegate District (at Large) (114th Congress),...",98,11,DC,DC-98-2016,90.91,7.28,90.86,4.09
7,379933,370995,38.5,750928,578965,98282,2120,17017,690667,17691,...,447,"Congressional District 1 (114th Congress), Flo...",01,12,FL,FL-01-2016,30.10,68.50,28.20,67.50
8,361387,356786,36.5,718173,497970,179656,2603,14334,664299,21475,...,645,"Congressional District 2 (114th Congress), Flo...",02,12,FL,FL-02-2016,34.00,64.60,30.60,66.20
9,358340,362765,37.9,721105,569233,98797,2150,23880,653621,25425,...,1889,"Congressional District 3 (114th Congress), Flo...",03,12,FL,FL-03-2016,41.90,56.60,40.20,56.20


In [16]:
#This line checks for any null values in the target column
#Some districts became obsolete between the 2012 and 2016 election, creating null values
election_census.Target_Presidential_R.isnull().sum(axis = 0)

14

In [17]:
#This line views any row with null values in the target column
election_census[election_census.isnull().any(axis=1)]

,Total_Males,Total_Females,Median_Age,Total_Pop,Total_White_People,Total_Afr_Am_People,Total_American_Ind_Alk_Ntv,Total_Asian,Total_Native_US,Total_Non_Citizens,...,Total_Dominican_Origin,District_Name,CD,State_Id,State,CD_Id_Year,Previous_Presidential_D,Previous_Presidential_R,Target_Presidential_D,Target_Presidential_R
355,1657467,1816715,40.0,3474182,2355164,341710,10984,7264,Replace,Replace,...,62452,Resident Commissioner District (at Large) (114...,98,72,PR,PR-98-2016,NaN,NaN,NaN,NaN
584,342099,339508,40.9,681607,633563,27775,1430,4709,667668,4822,...,75,"Congressional District 19 (112th Congress), Il...",19,17,IL,IL-19-2012,NaN,NaN,NaN,NaN
598,285053,291545,40.4,576598,537611,5372,3327,5593,550024,15637,...,0,"Congressional District 5 (112th Congress), Iowa",05,19,IA,IA-05-2012,NaN,NaN,NaN,NaN
615,331465,348427,35.7,679892,479335,175418,2675,7427,658222,11180,...,360,"Congressional District 7 (112th Congress), Lou...",07,22,LA,LA-07-2012,NaN,NaN,NaN,NaN
635,317401,339874,44.4,657275,594463,15443,2327,28413,592911,25300,...,1482,"Congressional District 10 (112th Congress), Ma...",10,25,MA,MA-10-2012,NaN,NaN,NaN,NaN
650,333426,347273,35.3,680699,519175,94577,2399,31799,615052,31930,...,337,"Congressional District 15 (112th Congress), Mi...",15,26,MI,MI-15-2012,NaN,NaN,NaN,NaN
671,340606,346201,37.2,686807,627385,29149,2521,10307,662713,10889,...,44,"Congressional District 9 (112th Congress), Mis...",09,29,MO,MO-09-2012,NaN,NaN,NaN,NaN
693,346106,346711,33.5,692817,403729,81090,1374,62188,364702,180334,...,61914,"Congressional District 13 (112th Congress), Ne...",13,34,NJ,NJ-13-2012,NaN,NaN,NaN,NaN
724,289423,317885,36.3,607308,376076,176549,3502,13318,554106,17145,...,1826,"Congressional District 28 (112th Congress), Ne...",28,36,NY,NY-28-2012,NaN,NaN,NaN,NaN
725,334295,331539,41.1,665834,606520,23153,4369,18203,625885,14716,...,856,"Congressional District 29 (112th Congress), Ne...",29,36,NY,NY-29-2012,NaN,NaN,NaN,NaN


In [18]:
#This line reassigns the dataframe to ensure no rows contain null values in the target column
election_census = election_census[pd.notnull(election_census['Target_Presidential_R'])]

In [19]:
election_census.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 0 to 872
Data columns (total 53 columns):
Total_Males                     860 non-null object
Total_Females                   860 non-null object
Median_Age                      860 non-null object
Total_Pop                       860 non-null object
Total_White_People              860 non-null object
Total_Afr_Am_People             860 non-null object
Total_American_Ind_Alk_Ntv      860 non-null object
Total_Asian                     860 non-null object
Total_Native_US                 860 non-null object
Total_Non_Citizens              860 non-null object
Total_Foreign_Born              860 non-null object
Total_Less_Highschool           860 non-null object
Total_Bachelor_Holders          860 non-null object
Median_Income                   860 non-null object
Total_Children                  860 non-null object
Total_Married_Households        860 non-null object
Total_STEM                      860 non-null object
Total_Busin

In [ ]:
#This line finds any cell that equals 'Replace' and prints the row
#election_census[election_census.eq('Replace').any(1)]

In [20]:
#Saves the main data frame
election_census.to_csv('congressional_data_and_targets.csv', index = False)